In [1]:
import numpy as np

from ortools.graph.python import min_cost_flow

In [2]:
start_nodes = [0, 1, 1, 2, 3]
end_nodes = [1, 2, 3, 4, 4]
capacities = [1, 1, 1, 1, 1]
costs = [0, 2, 1, 0, 0]

source = 0
sink = 4

supplies = [1, 0, 0, 0, -1]

In [3]:
smcf = min_cost_flow.SimpleMinCostFlow()

# Add each arc.
for i in range(len(start_nodes)):
    smcf.add_arc_with_capacity_and_unit_cost(
        start_nodes[i], end_nodes[i], capacities[i], costs[i]
    )
# Add node supplies.
for i in range(len(supplies)):
    smcf.set_node_supply(i, supplies[i])
    

In [4]:
status = smcf.solve()

if status == smcf.OPTIMAL:
    print("Total cost = ", smcf.optimal_cost())
    print()
    for arc in range(smcf.num_arcs()):
        # Can ignore arcs leading out of source or into sink.
        if smcf.tail(arc) != source and smcf.head(arc) != sink:
            # Arcs in the solution have a flow value of 1. Their start and end nodes
            # give an assignment of worker to task.
            if smcf.flow(arc) > 0:
                print(
                    "Worker %d assigned to task %d.  Cost = %d"
                    % (smcf.tail(arc), smcf.head(arc), smcf.unit_cost(arc))
                )
else:
    print("There was an issue with the min cost flow input.")
    print(f"Status: {status}")

Total cost =  1

Worker 1 assigned to task 3.  Cost = 1


In [37]:
def do_matching( relevant_task_data, num_tasks, num_resources, reduction = 0):
    start_nodes = [0 for i in range(num_resources)]
    end_nodes = [i+1 for i in range(num_resources)]
    capacities = [1 for i in range(num_resources)]
    costs = [0 for i in range(num_resources)]
    for task_ix, resource_ix, d in relevant_task_data:
        start_nodes += [resource_ix+1]
        end_nodes += [num_resources+1+task_ix]
        capacities += [1]
        costs += [d]

    start_nodes += [num_resources+1+task_ix for task_ix in range(num_tasks)]
    end_nodes += [num_tasks+num_resources+1 for task_ix in range(num_tasks)]
    capacities += [1 for i in range(num_tasks)]
    costs += [0 for i in range(num_tasks)]

    supplies = [min(num_tasks, num_resources) - reduction] +\
                [0 for i in range(num_tasks+num_resources)] +\
                [-min(num_tasks, num_resources) + reduction]

    print(start_nodes)
    print(end_nodes)
    print(capacities)
    print(costs)
    print(supplies)
    smcf = min_cost_flow.SimpleMinCostFlow()

    # Add each arc.
    for i in range(len(start_nodes)):
        smcf.add_arc_with_capacity_and_unit_cost(
            start_nodes[i], end_nodes[i], capacities[i], costs[i]
        )
    # Add node supplies.
    for i in range(len(supplies)):
        smcf.set_node_supply(i, supplies[i])

    status = smcf.solve_max_flow_with_min_cost()
    #status = smcf.solve()
    return smcf, status

In [38]:
relevant_task_data = [(0, 1, 333), (1, 1, 546)]
num_tasks = 2
num_resources = 2

smcf, status = do_matching(relevant_task_data, num_tasks, num_resources)

[0, 0, 2, 2, 3, 4]
[1, 2, 3, 4, 5, 5]
[1, 1, 1, 1, 1, 1]
[0, 0, 333, 546, 0, 0]
[2, 0, 0, 0, 0, -2]


In [39]:
print(status)
if status == smcf.OPTIMAL:
    print("Total cost = ", smcf.optimal_cost())
    print()
    for arc in range(smcf.num_arcs()):
        # Can ignore arcs leading out of source or into sink.
        if smcf.tail(arc) != source and smcf.head(arc) != sink:
            # Arcs in the solution have a flow value of 1. Their start and end nodes
            # give an assignment of worker to task.
            if smcf.flow(arc) > 0:
                print(
                    "Worker %d assigned to task %d.  Cost = %d"
                    % (smcf.tail(arc), smcf.head(arc), smcf.unit_cost(arc))
                )
else:
    print("There was an issue with the min cost flow input.")
    print(f"Status: {status}")

Status.INFEASIBLE
There was an issue with the min cost flow input.
Status: Status.INFEASIBLE
